In [104]:
require 'hdf5'

f = hdf5.open("data.hdf5", "r")

X_train = f:read("windows_train"):all()
Y_train = f:read("train_Y"):all()
X_valid = f:read("windows_valid"):all()
Y_valid = f:read("valid_reduced_Y"):all()

nclasses = f:read('nclasses'):all():long()[1]
nfeatures = f:read('nfeatures'):all():long()[1]

f:close()

In [114]:
function createHash(longTensor) 
    local s = ""
    for i=1,longTensor:size(1) do
        s = s .. tostring(longTensor[i]) .. " "
    end
    return s
end
    
function count_train(windows_train, space_train, n_gram)
    local count_table = {}
    local space_table = {}
    local restart = 1
    local windows_train
    local space_train
    for i=1,windows_train:size(1) do
        if restart < n_gram then
            for j=1,n_gram-restart do
                windows_train[i][j] = 30
            end
            restart = restart + 1
        end
        if windows_train[i][n_gram] == 30 then
            restart = 1
        end
        local key = createHash(windows_train[i])
        if count_table[key] then
            count_table[key] = count_table[key] + 1
        else
            count_table[key] = 1
            space_table[key] = 0
        end
        if space_train[i] == 2 then
            space_table[key] = space_table[key] + 1
        end
    end
    for k,_ in pairs(space_table) do
        space_table[k] = space_table[k]/count_table[k]
    end
    return space_table
end  

function count_predict(lookup_table, windows_valid, space_valid, n_gram)
    local restart = 1
    for i=1, 20 do
        if restart < n_gram then
            for j=1,n_gram-restart do
                windows_valid[i][j] = 30
            end
            restart = restart + 1
        end
        if windows_valid[i][n_gram] == 30 then
            restart = 1
        end
        local key = createHash(windows_valid[i])
        local prob = lookup_table[key]
        print(key, prob, Y_valid[i])
    end
end

In [115]:
win_size = 5
ngram_table = count(X_train, Y_train, win_size)

In [116]:
count_predict(ngram_table, X_valid, Y_valid, win_size)

30 30 30 30 2 	0	1	
30 30 30 2 14 	0	1	
30 30 2 14 15 	0	1	
30 2 14 15 18 	0	1	
2 14 15 18 20 	0	1	
14 15 18 20 12 	0	1	
15 18 20 12 4 	0	1	
18 20 12 4 19 	0.64864864864865	1	
20 12 4 19 18 	0.91666666666667	2	
12 4 19 18 28 	0	1	
4 19 18 28 12 	0	1	
19 18 28 12 1 	0	1	
18 28 12 1 24 	0.9047619047619	2	
28 12 1 24 28 	0	1	
12 1 24 28 22 	0	1	
1 24 28 22 1 	0	1	
24 28 22 1 15 	0	1	
28 22 1 15 21 	0.52857142857143	2	


22 1 15 21 28 	0	1	
1 15 21 28 21 	0	1	

